In [27]:
### Libraries ###

from bs4 import BeautifulSoup
import requests
import re

In [25]:
### Get monster list webpage html text ###

response = requests.get("https://www.dandwiki.com/wiki/5e_Monsters")

if response.status_code == 200:                                                 # Checks that the connection to the webpage was established
    monster_list_soup = BeautifulSoup(response.content, "html.parser")          # Makes the HTML soup from the content of the response and assigns it to a variable
    print(monster_list_soup.prettify())                                         # Prints content of the connected webpage
else:
    print(response)                                                             # If not a good connection, prints response code

Output hidden; open in https://colab.research.google.com to view.

In [35]:
### Put monster names into list from monster_list_soup ###

monster_href = []                                                               # List to store each monster's hyperlink

for a in monster_list_soup.find_all("a", href=re.compile("\(5e_Creature\)")):   # Finds all <a> tags that have a hyperlink to a monster. The '\' allows the parenthesis to be recognized
    monster_href.append("dandwiki.com" + a["href"])                             # Adds the web address prefix to the href attribute and appends it to the monster_href list

['dandwiki.com/wiki/Sea_Serpent_(5e_Creature)',
 'dandwiki.com/wiki/Young_Sea_Serpent,_Variant_(5e_Creature)',
 'dandwiki.com/wiki/Aaeaea_(5e_Creature)',
 'dandwiki.com/wiki/Aalar%27s_Chosen_Flesh_Golem_(5e_Creature)',
 'dandwiki.com/wiki/Aalar%27s_Chosen_Grimlock_(5e_Creature)',
 'dandwiki.com/wiki/Aarakocra_Galewing_(5e_Creature)',
 'dandwiki.com/wiki/Aarakocra_Skeleton_(5e_Creature)',
 'dandwiki.com/wiki/Aarakocra_Sky_Lord_(5e_Creature)',
 'dandwiki.com/wiki/Abaddon_(5e_Creature)',
 'dandwiki.com/wiki/Abaddon_the_Despoiler_(5e_Creature)',
 'dandwiki.com/wiki/Abaddon%27s_Locusts_(5e_Creature)',
 'dandwiki.com/wiki/Abathrael_(5e_Creature)',
 'dandwiki.com/wiki/Aberrant_Demagogue_(5e_Creature)',
 'dandwiki.com/wiki/Abigor_(5e_Creature)',
 'dandwiki.com/wiki/Abominable_Yeti_Mage_(5e_Creature)',
 'dandwiki.com/wiki/Abomination_of_Flesh_(5e_Creature)',
 'dandwiki.com/wiki/Abra_(5e_Creature)',
 'dandwiki.com/wiki/Absol_(5e_Creature)',
 'dandwiki.com/wiki/Abyssal_Dragon_(5e_Creature)',
 'da

In [ ]:
### Get webpage html text ###

response = requests.get("https://www.dandwiki.com/wiki/Aaeaea_(5e_Creature)")

if response.status_code == 200:                                         # Checks that the connection to the webpage was established
    Aaeaea_soup = BeautifulSoup(response.content, "html.parser")        # Makes the HTML soup from the content of the response and assigns it to variable MonsterName_soup
    print(Aaeaea_soup.prettify())                                       # Prints content of the connected webpage
else:
    print(response)                                                     # If not a good connection, prints response code

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Aaeaea (5e Creature) - D&amp;D Wiki
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"72d2b54c9e5b0640812ef31b","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Aaeaea_(5e_Creature)","wgTitle":"Aaeaea (5e Creature)","wgCurRevisionId":1651606,"wgRevisionId":1651606,"wgArticleId":218144,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CR1","5e","Creature","Undead Type","Tiny Size","Swamp Monster","User"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext